In [1]:
from pyspark.sql import SparkSession
import os

os.environ["SPARK_VERSION"] = '3.3'

In [2]:
!pip install pydeequ==1.5.0

In [3]:
import pydeequ
from pydeequ.analyzers import *

In [4]:
spark = SparkSession.builder \
    .appName("acura_dq") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.7.5.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord) \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres-datastore:5432/acura_db"
properties = {
    "user": "acura_user",
    "password": "password",
    "driver": "org.postgresql.Driver"
}
query = "(SELECT * FROM public_integration.dev_itg_dataeng_cc_review_master) AS reviews"
df = spark.read.jdbc(url=jdbc_url, table=query, properties=properties)

In [5]:
df.columns

['card_name',
 'platform_name',
 'review_timestamp',
 'review_id',
 'review_customer_id',
 'review_customer_name',
 'review_rating_maximum',
 'review_rating_given',
 'review_raw_text']

In [6]:
from pydeequ.profiles import *

In [18]:
analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("review_id")) \
                    .addAnalyzer(Distinctness("review_id")) \
                    .addAnalyzer(Mean("review_rating_given")) \
                    .addAnalyzer(Compliance("top review_rating_given", "star_rating >= 4.0")) \
                    .addAnalyzer(Correlation("platform_name", "review_rating_given")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult).toPandas()
analysisResult_df.head()

,entity,instance,name,value
0,Column,review_id,Distinctness,1.0


In [15]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Rating Metadata")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 3000000) \
        .hasMin("review_rating_given", lambda x: x == 1.0) \
        .hasMax("review_rating_given", lambda x: x == 5.0)  \
        .isComplete("review_id")  \
        .isUnique("review_id")  \
        .isComplete("card_name")  \
        .hasMin("review_year", lambda x: x == '1996') \
        .hasMax("review_year", lambda x: x == '2017')) \
    .run()
    
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark,checkResult, pandas=True)

/usr/local/spark/python/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [16]:
checkResult_df.head()

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Rating Metadata,Warning,Warning,SizeConstraint(Size(None)),Failure,Value: 140 does not meet the constraint requir...
1,Rating Metadata,Warning,Warning,"MinimumConstraint(Minimum(review_rating_given,...",Success,
2,Rating Metadata,Warning,Warning,"MaximumConstraint(Maximum(review_rating_given,...",Success,
3,Rating Metadata,Warning,Warning,"CompletenessConstraint(Completeness(review_id,...",Success,
4,Rating Metadata,Warning,Warning,UniquenessConstraint(Uniqueness(List(review_id...,Success,
